<a href="https://colab.research.google.com/github/KamilBienias/data-science/blob/main/training/analityk.edu.pl/pandas_tutorial/14_bazy_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
# https://analityk.edu.pl/python-pandas-tutorial-praca-z-bazami-danych-i-sql/

# **Python Pandas Tutorial – praca z bazami danych i SQL**

Do tej pory pracowaliśmy na danych pochodzących z plików, takich jak CSV czy XLS. Możliwości Pandas są jednak szersze. Bardzo dobrze współpracuje z bazami danych, których może odczytywać dane oraz może do nich zapisywać dane. Zobaczmy jak, na przykładnie bazy danych SQLite.
Na początek przygotujmy prosty DataFrame

In [131]:
import pandas as pd

df = pd.read_csv('https://drive.google.com/uc?export=download&id=1Ofob2EV42qPeJSpIlhxnsEMAdEvnop7l', 
                 sep=';', 
                 encoding = "ISO-8859-1",
                 skiprows=[1],
                 dtype={'Lenght':'float64', 'Popularity':'float64'},
                 usecols=['Year', 'Title', 'Length', 'Popularity', 'Subject', 'Awards'],
                 converters={'Awards':lambda x: True if x == 'Yes' else False})
df

,Year,Length,Title,Subject,Popularity,Awards
0,1990,111.0,Tie Me Up! Tie Me Down!,Comedy,68.0,False
1,1991,113.0,High Heels,Comedy,68.0,False
2,1983,104.0,"Dead Zone, The",Horror,79.0,False
3,1979,122.0,Cuba,Action,6.0,False
4,1978,94.0,Days of Heaven,Drama,14.0,False
...,...,...,...,...,...,...
1654,1932,226.0,"Shadow of the Eagle, The",Action,19.0,False
1655,1989,103.0,Blood & Guns,Action,43.0,False
1656,1988,78.0,Hot Money,Drama,19.0,False
1657,1977,75.0,Comedy Tonight,Comedy,18.0,False


In [132]:
df.dtypes

Year            int64
Length        float64
Title          object
Subject        object
Popularity    float64
Awards           bool
dtype: object

## **Zapis do bazy danych SQLite**

Aby zapisać dane z naszego DataFrame df, do bazy SQLite, process jest bardzo prosty.

Importujemy bibliotekę sqlite3, otwieramy połączenie do bazy danych. Jeżeli nie istnieje, to zostanie stworzona. A następnie za pomocą funkcji to_sql zapisujemy dane:

In [133]:
import sqlite3 as sql
import os

# można podać jawnie ścieżkę
# conn = sql.connect('c:/python/pandas/nazwa_bazy.sqlite')

# sam tworzę ścieżkę
base_path = os.path.join('baza_sqlite3')
if not os.path.isdir(base_path):
        os.makedirs(base_path)

conn = sql.connect(os.path.join(base_path, 'nazwa_bazy.sqlite'))

# zapisuje df do bazy new.sqlite jako tabelę o nazwie films
df.to_sql('films', conn)

## **Odczyt z bazy danych SQLite**

Równie prosty jest odczyt danych z bazy danych. Użyjemy funkcji read_sql, natomiast pierwszym parametrem będzie kod SQL który chcemy wykonać i którego wyniki chcemy zapisać do naszego DataFrame:



In [134]:
# można podać jawnie ścieżkę
# conn = sql.connect('c:/python/pandas/nazwa_bazy.sqlite')

conn = sql.connect(os.path.join(base_path, 'nazwa_bazy.sqlite'))
movies_later_than_1995 = pd.read_sql("select * from films where Year > 1995", conn)
movies_later_than_1995

,index,Year,Length,Title,Subject,Popularity,Awards
0,81,1996,96.0,"Island of Dr. Moreau, The",Horror,39.0,0
1,1209,1997,109.0,Alien: resurrection,Science Fiction,60.0,0


## **Mój trening**

In [135]:
# cała tabela films uporządkowana po roku (zamieniło False na 0 i True na 1)
# whole_table = pd.read_sql("SELECT * FROM films ORDER BY Year", conn)
whole_table = pd.read_sql("SELECT * FROM films", conn)
whole_table

,index,Year,Length,Title,Subject,Popularity,Awards
0,0,1990,111.0,Tie Me Up! Tie Me Down!,Comedy,68.0,0
1,1,1991,113.0,High Heels,Comedy,68.0,0
2,2,1983,104.0,"Dead Zone, The",Horror,79.0,0
3,3,1979,122.0,Cuba,Action,6.0,0
4,4,1978,94.0,Days of Heaven,Drama,14.0,0
...,...,...,...,...,...,...,...
1654,1654,1932,226.0,"Shadow of the Eagle, The",Action,19.0,0
1655,1655,1989,103.0,Blood & Guns,Action,43.0,0
1656,1656,1988,78.0,Hot Money,Drama,19.0,0
1657,1657,1977,75.0,Comedy Tonight,Comedy,18.0,0


In [136]:
# wstawiam nowy wiersz, ale jest błąd: if chunksize is not None: TypeError: 'NoneType' object is not iterable
# pd.read_sql("INSERT INTO films (Year, Length, Title, Subject, Popularity) VALUES (1986, 98, 'Krótkie spięcie', 'Science Fiction', 80)", conn)
# conn.commit()

# sprawdzam filmy science fiction z 1986
films_from_1986 = pd.read_sql("""SELECT * FROM films 
    WHERE Year == 1986 AND Subject == 'Science Fiction' """, conn)
films_from_1986

,index,Year,Length,Title,Subject,Popularity,Awards
0,507,1986,110.0,Highlander,Science Fiction,8.0,0
1,1031,1986,89.0,Legend,Science Fiction,42.0,0
2,1207,1986,137.0,Aliens,Science Fiction,82.0,0
